<a href="https://colab.research.google.com/github/gkrry2723/ML_pytorch_study/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**CNN**

##cnn mnist




In [18]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
import torch.nn as nn



In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
  torch.cuda.manual_seed_all(777)

In [9]:
#parameters
learning_rate=0.001
training_epochs=15
batch_size=100

In [14]:
#MNIST dataset

mnist_train = dsets.MNIST(root='MNIST_data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                          train = False,
                          transform = transforms.ToTensor(),
                          download=True)


In [16]:
#data loader 만들기
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True
                                         )

In [21]:
#model 만들기
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    self.layer1=nn.Sequential(
        nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    self.layer2=nn.Sequential(
        nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )


    self.fc = nn.Linear(7*7*64, 10, bias=True)
    torch.nn.init.xavier_uniform_(self.fc.weight)
  
  def forward(self,x):
    out = self.layer1(x)
    out = self.layer2(out)

    out = out.view(out.size(0),-1)
    out = self.fc(out)
    return out


In [22]:
model=CNN().to(device)

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)

In [24]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [26]:
#training
total_batch = len(data_loader)
print("Learning Started.")

for epoch in range(training_epochs):
  avg_cost = 0

  for X,Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost +=cost/total_batch

  print('[Epoch:{}] cost = {}'.format(epoch+1,avg_cost))

print('Learning Fin')

Learning Started.
[Epoch:1] cost = 0.0159787368029356
[Epoch:2] cost = 0.012473936192691326
[Epoch:3] cost = 0.011062804609537125
[Epoch:4] cost = 0.010011661797761917
[Epoch:5] cost = 0.008859047666192055
[Epoch:6] cost = 0.007630601525306702
[Epoch:7] cost = 0.006760051473975182
[Epoch:8] cost = 0.005533881019800901
[Epoch:9] cost = 0.004860235843807459
[Epoch:10] cost = 0.0042067039757966995
[Epoch:11] cost = 0.0038668885827064514
[Epoch:12] cost = 0.004979636520147324
[Epoch:13] cost = 0.003371435683220625
[Epoch:14] cost = 0.00313518731854856
[Epoch:15] cost = 0.004945240914821625
Learning Fin


In [27]:
#test

with torch.no_grad():
  X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:',accuracy.item())

Accuracy: 0.9866999983787537


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
